In [1]:
import pandas as pd
import numpy as np
import requests
from PIL import Image
import io
import time
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
from tqdm import tqdm

In [2]:
image_df = pd.read_csv('postcard_data.tsv', sep='\t')
image_df.head()

,object_id,title,guid,ids_id
0,edanmdm-nmah_1888444,Two Men and a Woman Congregating Downtown,http://n2t.net/ark:/65665/ng49ca746b3-d555-704...,NMAH-AHB2018q010241
1,edanmdm-nmah_1888444,Two Men and a Woman Congregating Downtown,http://n2t.net/ark:/65665/ng49ca746b3-d555-704...,NMAH-AHB2018q010242
2,edanmdm-nmah_1888447,Portrait of Couple Standing Outside Cloth Tent,http://n2t.net/ark:/65665/ng49ca746b3-d558-704...,NMAH-AHB2018q010352
3,edanmdm-nmah_1888447,Portrait of Couple Standing Outside Cloth Tent,http://n2t.net/ark:/65665/ng49ca746b3-d558-704...,NMAH-AHB2018q010395
4,edanmdm-nmah_1888447,Portrait of Couple Standing Outside Cloth Tent,http://n2t.net/ark:/65665/ng49ca746b3-d558-704...,NMAH-AHB2018q010396


In [3]:
ids_list = image_df['ids_id'].tolist()
ids_list[:5]

['NMAH-AHB2018q010241',
 'NMAH-AHB2018q010242',
 'NMAH-AHB2018q010352',
 'NMAH-AHB2018q010395',
 'NMAH-AHB2018q010396']

In [4]:
def requests_PIL_download(ids_id):
    width, height = np.nan, np.nan
    image_url = f'https://ids.si.edu/ids/deliveryService/id/{ids_id}/500'
    filename = 'images/{}.jpg'.format(ids_id)
    r = requests.get(image_url, timeout=60)
    if r.headers['Content-Type'] == 'image/jpeg':
        with Image.open(io.BytesIO(r.content)) as im:
            width, height = im.size
            im.save(filename)
    return {'width': width, 'height': height, 'ids_id': ids_id}

In [5]:
requests_PIL_download('NMAH-AHB2018q010241')

{'width': 339, 'height': 499, 'ids_id': 'NMAH-AHB2018q010241'}

In [6]:
start_time = time.perf_counter()

dimension_list = []

with ThreadPoolExecutor(max_workers=6) as executor:
    dimension_list = list(tqdm(executor.map(requests_PIL_download, ids_list),
                               total = len(ids_list)))

end_time = time.perf_counter()
elapsed_time = end_time - start_time

print(f"Downloaded {len(dimension_list)} images in {elapsed_time} s")

100%|██████████████████████████████████████████████████████████████████████████████| 2863/2863 [01:24<00:00, 33.73it/s]

Downloaded 2863 images in 84.97528930000044 s
